In [ ]:
# %pip install tqdm

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, BatchNormalization, Activation, Input, Dense, Flatten, Dropout, Concatenate
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

import numpy as np
from tqdm import tqdm

import pickle


import os

# Configuration

In [47]:
data_path = '../data/MIT-BIH/'
minute_input = 10
minute_output = 10
window_input= 40*minute_input
window_out= 40*minute_input
train_size = 0.8
test_size = 1 - train_size
data_set = {
  0: "test",
  1: "train"
}

# data_set = {
#   0: "test_ram",
#   1: "train_ram"
# }

# length_ecg là độ dài 2 khoảng RR được fixed lúc tiền xử lý dữ liệu 
# (độ dài một dòng trong file excel, trừ cột cuối là nhãn lớp bệnh tim)
length_ecg = 187 
batch_size = 16
model_cls = "LSTM"
'''
Độ dài của input/output càng dài thì số lượng file không đáp ứng đủ để tạo một mẫu hợp lệ 
cho mô hình càng nhiều. Để đám bảo tính thống nhất nên sẽ dùng độ dài dài nhất của phần 
input/output trong quá trình thực nghiệm để là chuẩn từ đó loại các file bị thiếu này đều
ở những phần thực nghiệm input/output khác.
'''
missing_file_train = ['201_V1.csv', '102_V2.csv', '124_V4.csv', '112_V1.csv', '203_V1.csv', '116_V1.csv', '108_V1.csv', '207_V1.csv', '111_V1.csv', '200_V1.csv', '207_MLII.csv', '210_V1.csv', '202_V1.csv', '113_V1.csv', '214_V1.csv', '121_V1.csv', '109_V1.csv', '105_V1.csv', '107_V1.csv', '115_V1.csv', '208_V1.csv']
missing_file_test = ['213_V1.csv', '231_V1.csv', '228_V1.csv', '222_V1.csv', '232_V1.csv']

# Training Regression

## Get data

In [48]:
def get_data(istrainset):    
    missing_file = []
    total_sample = 0
    directory = f"{data_path}{data_set[istrainset]}/"
    X, y = [], []
    for filename in os.listdir(directory):
        if (filename not in (missing_file_train)) and (filename not in (missing_file_test)) and (filename != ".DS_Store"):
            f = os.path.join(directory, filename)
            if os.path.isfile(f):
                df = pd.read_csv(f, header=None)
                data=df.drop(columns=length_ecg)
                data=data.values
                data1=df.iloc[:, length_ecg]
                # Số lượng lặp qua dữ liệu
                num_samples = len(data) - window_input - 1 - window_out
                # Tạo dữ liệu train từ cửa sổ trượt
                if num_samples > 0:
                    total_sample = total_sample + num_samples
                    for i in range(num_samples):
                        X_window = data[i:i+window_input]
                        y_value = data1[i+window_input+window_out]

                        X.append(X_window)
                        y.append(y_value)
                else:
                    missing_file.append(filename)
    print("------🍒------")
    print(f"Num of file in {data_set[istrainset]} set can not use due to its missing of length: {len(missing_file)}")
    print(f"Number of sample: {len(y)}/{len(X)}/{total_sample}")
    print(f"Missing files: {missing_file}")
    return X,y
   

In [49]:
# # Lấy tập train/test
X_train, y_train = get_data(1)
X_test, y_test = get_data(0)

------🍒------
Num of file in train set can not use due to its missing of length: 0
Number of sample: 69014/69014/69014
Missing files: []
------🍒------
Num of file in test set can not use due to its missing of length: 0
Number of sample: 19782/19782/19782
Missing files: []


In [50]:
class Dataset:
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        # read data
        X = self.data[i]
        y = self.label[i].astype(int)
        return X, y

class Dataloader(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size,size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.size= size

    def __getitem__(self, i):
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        return tuple(batch)

    def __len__(self):
        return self.size //self.batch_size
    
train_dataset = Dataset(X_train, y_train)
test_dataset = Dataset(X_test, y_test)
train_loader = Dataloader(train_dataset, batch_size,len(train_dataset))
test_loader = Dataloader(test_dataset,batch_size,len(test_dataset))

In [73]:
X_train[0].shape

(400, 187)

In [51]:
train_dataset = Dataset(X_train, y_train)
test_dataset = Dataset(X_test, y_test)

In [52]:
train_loader = Dataloader(train_dataset, batch_size,len(train_dataset))
test_loader = Dataloader(test_dataset,batch_size,len(test_dataset))

## Build Model

In [53]:
def residual_block(x, filters, kernel_size=3, dilation_rate=1):
    # Convolutional block with batch normalization and ReLU activation
    x_res = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding='same')(x)
    x_res = BatchNormalization()(x_res)
    x_res = Activation('relu')(x_res)

    # Residual connection
    x_res = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding='same')(x_res)
    x_res = BatchNormalization()(x_res)
    x_res = Activation('relu')(x_res)

    x = tf.keras.layers.Add()([x, x_res])
    return x

In [54]:
def first_branch_extract_ecg():
    inputs = Input(shape=(window_input, length_ecg))

    # Convolutional block
    x = Conv1D(64, kernel_size=3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Residual blocks
    for _ in range(3):  # You can adjust the number of residual blocks
        x = residual_block(x, filters=64, kernel_size=3)

    # Global average pooling
    x = tf.keras.layers.GlobalAveragePooling1D()(x)

    x = Flatten()(x)
    return x

In [55]:
def second_branch_extract_arrhythmia(model_name):
    if model_name == "CNN":
        inputs = Input(shape=(window_input, length_ecg))

        # Convolutional block
        x = Conv1D(64, kernel_size=3, padding='same')(inputs)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        # Residual blocks
        for _ in range(3):  # You can adjust the number of residual blocks
            x = residual_block(x, filters=64, kernel_size=3)

        # Global average pooling
        x = tf.keras.layers.GlobalAveragePooling1D()(x)

        x = Flatten()(x)
    elif model_name == "LSTM":
        input_layer = Input(shape=(window_input, length_ecg))
        x = LSTM(64, activation='relu', return_sequences=True)(input_layer)
        x = Dropout(0.2)(x)
        x = LSTM(64, activation='relu', return_sequences=True)(x)
        x = Dropout(0.2)(x)
        x = LSTM(64, activation='relu')(x)
        x = Dropout(0.2)(x)
        x = Flatten()(x)
    return x

In [88]:
def build_model_cls(model_name_arrbranch):
    # First extract ECG branch
    x_first_branch_input = Input(shape=(window_input, length_ecg))
    # Convolutional block
    x_first_branch = Conv1D(64, kernel_size=3, padding='same')(x_first_branch_input)
    x_first_branch = BatchNormalization()(x_first_branch)
    x_first_branch = Activation('relu')(x_first_branch)
    # Residual blocks
    for _ in range(3):  # You can adjust the number of residual blocks
        x_first_branch = residual_block(x_first_branch, filters=64, kernel_size=3)
    # Global average pooling
    x_first_branch = tf.keras.layers.GlobalAveragePooling1D()(x_first_branch)
    x_first_branch = Flatten()(x_first_branch)
    x_first_branch = tf.keras.Model(inputs=x_first_branch_input, outputs=x_first_branch)

    # Second extract Arrhythmia branch
    if model_name_arrbranch == "LSTM":
        x_second_branch_input = Input(shape=(window_input, length_ecg))
        x_second_branch = LSTM(64, activation='relu', return_sequences=True)(x_second_branch_input)
        x_second_branch = Dropout(0.2)(x_second_branch)
        x_second_branch = LSTM(64, activation='relu', return_sequences=True)(x_second_branch)
        x_second_branch = Dropout(0.2)(x_second_branch)
        x_second_branch = LSTM(64, activation='relu')(x_second_branch)
        x_second_branch = Dropout(0.2)(x_second_branch)
        x_second_branch = Flatten()(x_second_branch)
        x_second_branch = tf.keras.Model(inputs=x_second_branch_input, outputs=x_second_branch)
    if model_name_arrbranch == "CNN":
        x_second_branch_input = Input(shape=(window_input, length_ecg))

        # Convolutional block
        x_second_branch = Conv1D(64, kernel_size=3, padding='same')(x_second_branch_input)
        x_second_branch = BatchNormalization()(x_second_branch)
        x_second_branch = Activation('relu')(x_second_branch)
        # Residual blocks
        for _ in range(3):  # You can adjust the number of residual blocks
            x_second_branch = residual_block(x_second_branch, filters=64, kernel_size=3)
        # Global average pooling
        x_second_branch = tf.keras.layers.GlobalAveragePooling1D()(x_second_branch)
        x_second_branch = Flatten()(x_second_branch)
        x_second_branch = tf.keras.Model(inputs=x_second_branch_input, outputs=x_second_branch)

    concatenated = Concatenate()([x_first_branch.output, x_second_branch.output])
    
    x = Dense(64, activation='relu')(concatenated)
    x = Dense(32, activation='relu')(x)


    # Output layer for regression (linear activation)
    outputs = Dense(5, activation='softmax')(x)

    learning_rate = 0.001
    # Create and compile the model
    model = tf.keras.Model(inputs=[x_first_branch.input,x_second_branch.input], outputs=outputs)
    adam = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)    
    model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

## Training

In [79]:
class CombinedDataset:
    def __init__(self, data1, data2, label):
        self.data1 = data1
        self.data2 = data2
        self.label = label

    def __len__(self):
        return len(self.label)

    def __getitem__(self, i):
        X1 = self.data1[i]
        X2 = self.data2[i]
        y = self.label[i].astype(int)
        return X1, X2, y

class CombinedDataLoader(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size, size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.size = size

    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size

        data = [self.dataset[j] for j in range(start, stop)]

        X1_batch = np.stack([sample[0] for sample in data], axis=0)
        X2_batch = np.stack([sample[1] for sample in data], axis=0)
        y_batch = np.array([sample[2] for sample in data])

        return [X1_batch, X2_batch], y_batch

    def __len__(self):
        return self.size // self.batch_size

# Create the combined dataset
combined_train_dataset = CombinedDataset(X_train, X_train, y_train)
combined_train_loader = CombinedDataLoader(combined_train_dataset, batch_size, len(combined_train_dataset))

# Create the combined dataset
combined_test_dataset = CombinedDataset(X_test, X_test, y_test)
combined_test_loader = CombinedDataLoader(combined_test_dataset, batch_size, len(combined_test_dataset))


In [89]:
model_name_arrbranch = "CNN"
epochs = 2
model = build_model_cls(model_name_arrbranch)
# train_generator = combined_data_generator(train_loader, train_loader)
# model.fit(train_generator, validation_data=test_loader, verbose=1, epochs=epochs)
# model.save(f"trained/New_Proposed_CNN_{model_name_arrbranch}_PastECG_FutureCls_{minute_input}-mininput_{minute_output}-minoutput.h5")


In [90]:
tf.config.run_functions_eagerly(True)
model.fit(combined_train_loader, validation_data=combined_test_loader,verbose=1, epochs=epochs)

Epoch 1/2


/Users/trinhtruc/Library/Python/3.9/lib/python/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


  47/4313 [..............................] - ETA: 9:14 - loss: 0.7980 - accuracy: 0.8191ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/Users/trinhtruc/Library/Python/3.9/lib/python/site-packages/keras/src/backend.py", line 5159, in <genexpr>
    ta.write(ta_index_to_write, out)  File "/Users/trinhtruc/Library/Python/3.9/lib/python/site-packages/tensorflow/python/util/tf_should_use.py", line 288, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),
4313/4313 [==============================] - 572s 133ms/step - loss: 0.2517 - accuracy: 0.9448 - val_loss: 1.4492 - val_accuracy: 0.7904
Epoch 2/2
4313/4313 [==============================] - 567s 131ms/step - loss: 0.1856 - accuracy: 0.9571 - val_loss: 2.4998 - val_accuracy: 0.7270


In [ ]:
model.summary()

In [ ]:
model_name = "LSTM"
epochs = 2
model = build_model_cls(model_name, epochs = epochs)
model.fit(train_loader, validation_data=test_loader, verbose=1, epochs=epochs)

model.fit([train_loader, train_loader], epochs=10, batch_size=32)

model.save(f"trained/{model_name}_PastECG_FutureCls_{minute_input}-mininput_{minute_output}-minoutput.h5")
print("Saved model to disk")

In [92]:
x = np.load("X_test_1_1.npy")
y = np.load("y_test_1_1.npy")
print(x.shape)
print(y.shape)

(30582, 7480)
(30582,)


In [94]:
x = np.load("/Users/trinhtruc/X_test_10_1.npy")
# y = np.load("y_train_1_1.npy")
print(x.shape)
# print(y.shape)